In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import tensorflow as tf
import numpy as np
import time
import json

from model import Caption_Model
from data_generator import Data_Generator
from inference_util import Inference

import inception_base
import configuration

FLAGS = tf.app.flags.FLAGS

tf.flags.DEFINE_integer("batch_size", 64,
                       "Batch size of train data input.")
tf.flags.DEFINE_integer("beam_size", 3,
                       "Beam size.")
tf.flags.DEFINE_string("checkpoint_model", None,
                       "Model Checkpoint to use.")
tf.flags.DEFINE_integer("max_captions", None,
                       "Maximum number of captions to generate")
tf.flags.DEFINE_integer("max_len_captions", None,
                       "Maximum length of captions to generate")
tf.flags.DEFINE_string("dataset", "test",
                       "Dataset to use")
tf.flags.DEFINE_string("outfile_name", "generated_caption.json",
                       "Name of the output result file")

In [ ]:
data_config = configuration.DataConfig().config

In [ ]:
model_path = tf.train.latest_checkpoint(data_config["checkpoint_dir"])

In [ ]:
model_paths=[model_path]
dataset="test"
batch_size=64
max_len_captions=20
max_captions=None
beam_size=3

In [ ]:
data_config = configuration.DataConfig().config
data_gen = Data_Generator(processed_video_dir = data_config["processed_video_dir"],
                        caption_file = data_config["caption_file"],
                        unique_freq_cutoff = data_config["unique_frequency_cutoff"],
                        max_caption_len = data_config["max_caption_length"])

data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])

assert dataset in ["val","test","train"]

if max_len_captions:
    max_len = max_len_captions
else:
    max_len = data_config['max_caption_length']

model_config = configuration.ModelConfig(data_gen).config
model = Caption_Model( **model_config,mode="inference")
model.build()

In [ ]:
infer_util = Inference(model,data_gen.word_to_idx,data_gen.idx_to_word)

if max_captions:
    max_iter = max_captions
else:
    max_iter = len(data_gen.dataset[dataset])+10 #+10 is just to be safe ;)

video_paths = {i["file_name"]:i["path"] for i in data_gen.dataset[dataset]}


In [ ]:
gen_captions = []
sess = tf.Session()
model_path = model_paths[0]

In [ ]:
gen_caption = []    
saver = tf.train.Saver()

if model_path != None:
    print("Restoring weights from %s" %model_path)
    saver.restore(sess,model_path)

else:
    print("No checkpoint found. Exiting")

video_files = list(video_paths.keys())    

iter = 0
btch = 0

In [ ]:
start = btch
end = min(len(video_files),btch+batch_size)
dataset={}
dataset["video"] = np.asarray([np.load(video_paths[video_files[i]]) for i in range(start,end)])
dataset["path"] = [video_paths[video_files[i]] for i in range(start,end)]
dataset["file"] = [video_files[i] for i in range(start,end)]

In [ ]:
dataset["gen_caption"] = infer_util.generate_caption_batch_beam(sess,dataset["video"])

In [ ]:
for i in range(len(dataset['gen_caption'])):
    dictionary = {}
    dictionary["gen_caption"] = dataset['gen_caption'][i]
    dictionary["file_name"] = dataset['file'][i]
    dictionary["path"] = dataset['path'][i]
    gen_caption.append(dictionary)
    iter+=1
    if iter >= max_iter:
        break